<a href="https://colab.research.google.com/github/peterbmob/DHMVADoE/blob/main/Excercises/class_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analysis of DoE
In this jupyter book, we use the data from the video and perform the analysis with our python tools.

We startwith importing the needed modules, followed by loading the data.

In [ ]:
import pandas as pd
import numpy as np

data=pd.read_excel('data_example.xlsx', index_col=0, header=1)
data

Since the excel file we loaded the data from included empty rows, we need to clean it. this can be done using the drop function in pandas.

In [ ]:
data=data.drop(['Unnamed: 4', 'Unnamed: 5', 'Unnamed: 11'], axis=1)
data

In the video, the DoE was alreay performed, so he already had the design matrix. Let's do it, it makes it prettier and easier to follow.First, let's find the span and label of each variable:

In [ ]:
# create dictionary for parameters
inputs_labels = {'A' : data.columns[0],
                 'B' :  data.columns[1],
                 'C' :  data.columns[2]}

#create list of data for high and low.
dat = [ ('A',data.loc[:,'Time'].min(),data.loc[:,'Time'].max()),
        ('B',data.loc[:,'Temp'].min(),data.loc[:,'Temp'].max()),
        ('C',data.loc[:,'Pressure'].min(),data.loc[:,'Pressure'].max())]

# create pandas dataframe in a pandas dataframe
inputs_df = pd.DataFrame(dat,columns=['index','low','high'])
inputs_df = inputs_df.set_index(['index'])
inputs_df['label'] = inputs_df.index.map( lambda z : inputs_labels[z] )

#print dataframe
inputs_df

and encode the data usng the span and the averages:

In [ ]:
# compute averages and span
inputs_df['average'] = inputs_df.apply( lambda z : ( z['high'] + z['low'])/2 , axis=1)
inputs_df['span'] = inputs_df.apply( lambda z : ( z['high'] - z['low'])/2 , axis=1)

# encode the data
inputs_df['encoded_low'] = inputs_df.apply( lambda z : ( z['low']  - z['average'] )/( z['span'] ), axis=1)
inputs_df['encoded_high'] = inputs_df.apply( lambda z : ( z['high'] - z['average'] )/( z['span'] ), axis=1)

inputs_df = inputs_df.drop(['average','span'],axis=1)

inputs_df

and generate the design matrix:

In [ ]:
import itertools
encoded_inputs= list(itertools.product([-1,1],[-1,1],[-1,1]))
encoded_inputs

In [ ]:
results=pd.DataFrame(encoded_inputs)
results=results[results.columns[::-1]]
results.columns=['A','B','C']
results

which gives the following matrix for the experiments to be performed:

In [ ]:
real_experiment = results

var_labels = []
for var in ['A','B','C']:
    var_label = inputs_df.loc[var]['label']
    var_labels.append(var_label)
    real_experiment[var_label] = results.apply(
        lambda z : inputs_df.loc[var]['low'] if z[var]<0 else inputs_df.loc[var]['high'] ,
        axis=1)

print("The values of each real variable in the experiment:")
real_experiment[var_labels]

Note that the order is changed from the minitab example. However, this will not the results. The choice is upt to you, I prefer to divide it like this :).

## 2. Look us also have closer look at response

In the data table, the average y and standard deviation is already given... however, if they are not we can easily compute them using pandas and numpy.

In [ ]:
y=data.loc[:,'Y1':'Y5']
# We already had the average y and standard deviation in the table, if not, these are easily accessible pandas and numpy.
y_mean=y.mean(axis=1)
y_std = y.std(axis=1)
results['y']=y_mean.to_list()
results['s']=y_std.to_list()
results

These values will of course be the same as the one in the original data table

In [ ]:
data[['Average','Sigma']]

## 3. Let's analyse the results
In this example we will use statmodels and its "fit to formula function" to analyze the results. Since we are looking at the column for the average the full model since then we will not have any degrees of freedom to make a statistical analysis of our model. It is still possibble to perform the OLS with the full model; you just won't get any statistics for your model (test this yourself).

How to reduce the model?
Here, we often use domain expertise... and expercience. In physics and chemistry, a general rule of thumb is that higher order intercations can be neglected. In this first shot, we therefore remove the **ABC** interaction.

We start looking at the standard variation:

In [ ]:
import statsmodels.formula.api as smf


dat=results.loc[:,'A':'C']
dat['s']=results.loc[:,'s']

mod = smf.ols(formula='s ~ A + B + C + A:B + A:C + B:C', data=dat)

res = mod.fit()
res.summary()

As we see, none of the terms are significant (high p-values) indicating that the model is not of any use for us.


Lets turn to the response (y):

In [ ]:
import statsmodels.formula.api as smf


dat=results.loc[:,'A':'C']
dat['y']=results.loc[:,'y']

mod = smf.ols(formula='y ~ A + B + C + A:B + A:C + B:C', data=dat)

res = mod.fit()
res.summary()

Note: test to add the **ABC** interaction. When we use the full model, we do not have any Degrees of freedom to evaluate residuals, i.e. **Df Residuals:	0**. But we can still use this data to analyze the standardized effects of our variables and interactions between variables. Don't forget to reduce the model before you continue with the following boxes.




In the model for y we have A and B that show p-values smaller than 0.05, which means that they significantly contribute to the model. The rest are larger and are thus not significant.

This can also bee analyzed in a graphical form. To do this, we summarize the data and plot the results in a **Pareto chart**. This is done by computing the standardized effects and the cumulative percentage of standardized effects, i.e. the percentage of how much of the total effect each individual effect contributes with.  

In [ ]:
effects=pd.DataFrame()

# the standardised effects are the absolute values of the coefficients obtained from the OLS.
effects['Standardized effect']=pd.DataFrame(np.abs(res.params[1:]))

# Add cumulative percentage column.
effects["cum_percentage"] = round(effects["Standardized effect"].cumsum()/effects["Standardized effect"].sum()*100,2)
effects

This is best illustrated using a results in a Pareto chart.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
plt.rcParams.update({'font.size': 22})
# Set figure and axis
fig, ax = plt.subplots(figsize=(22,10))

# Plot bars (i.e. frequencies)
ax.set_title("Pareto Chart")
ax.set_xlabel("Parameter")
ax.set_ylabel("Frequency");
effects['Standardized effect'].plot.bar(ax=ax)

# Second y axis (i.e. cumulative percentage)
ax2 = ax.twinx()
ax2.plot(effects.index, effects["cum_percentage"], color="red", marker="D", ms=7)
effects.plot(y="cum_percentage", color="red", marker="D", ms=7, ax=ax2)
ax2.yaxis.set_major_formatter(PercentFormatter())
ax2.set_ylabel("Cumulative Percentage");
ax2.axhline(80, color="orange", linestyle="dashed")


The orange line is the Pareto Principle of *"Vital few"* and *"Trivial many"*, also called the 80/20 tule. The “vital few” are contributing to 80% of the problem, therefore, they should take the highest priority when determining areas to improve as they will give you the “biggest bang for your buck”. Since the “trivial many” only represent 20% of the problem, you will not see much improvement if you focus on these areas. This is the idea behind the 80/20 rule which we will dive deeper into below.

To better illustrate the results, it might be wise to sort the data. This can be done with the **sort_values** function in pandas.



In [ ]:
effects_sorted = effects.sort_values(by='Standardized effect', ascending=False)
effects_sorted

and be visualized in a Pareto chart

In [ ]:
# Add cumulative percentage column.
effects_sorted["cum_percentage"] = round(effects_sorted["Standardized effect"].cumsum()/effects_sorted["Standardized effect"].sum()*100,2)

# and plot Pareto Chart
plt.rcParams.update({'font.size': 22})
# Set figure and axis
fig, ax = plt.subplots(figsize=(22,10))

# Plot bars (i.e. frequencies)
ax.set_title("Pareto Chart")
ax.set_xlabel("Parameter")
ax.set_ylabel("Frequency");
effects_sorted['Standardized effect'].plot.bar(ax=ax)

# Second y axis (i.e. cumulative percentage)
ax2 = ax.twinx()
ax2.plot(effects_sorted.index, effects_sorted["cum_percentage"], color="red", marker="D", ms=7)
effects_sorted.plot(y="cum_percentage", color="red", marker="D", ms=7, ax=ax2)
ax2.yaxis.set_major_formatter(PercentFormatter())
ax2.set_ylabel("Cumulative Percentage");
ax2.axhline(80, color="orange", linestyle="dashed")


Here we clearly se that A and C are the important parameters in our model. Let's use them and create the final model.

In [ ]:
mod = smf.ols(formula='y ~ A + C', data=dat)

res = mod.fit()
res.summary()

Now we have a model, and we can ask it to give test the model... Let's start with the real data...

In [ ]:
results['y']

In [ ]:
import matplotlib.pyplot as plt
ypred = res.predict(results.loc[:,'A':'C'])
#print(ypred)

d = pd.DataFrame({'y_pred':ypred,'y_real':results['y']})
d.plot.scatter(x='y_real', y='y_pred')

x=np.linspace(-5,160,101)

plt.plot(x,x,'k-') # identity line